In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import git
import mlflow.keras
import os, sys
import numpy as np
from gensim.models.wrappers import FastText

np.random.seed(42)
import tensorflow_hub as hub
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.regularizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.preprocessing import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import *

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
%matplotlib inline
pd.set_option('display.max_rows', None)

sys.path.append('../citation_bio_trainer')
#from feature.SpacyFeaturizer import get_spacy_feats_from_text
from feature.Featurizer import Featurizer
from util.Utils import calulate_ser_jer, load_from_folder, pad_sequences, load_embedding_matrix, evaluate, log_mlflow_results
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/joshib/.cache/pypoetry/virtualenvs/dp-dpo-citation-bio-trainer-FaGiNfZ9-py3.7/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [4]:
path = '/home/joshib/cs_data/citation-bio-labelled-data-2020-06-08~15:13:57.766608/nlp/exps/output/2020-06-08~15:13:57.766608'
#data_df = pd.read_csv(path+'/data-2020-06-08~15:14:01.185827.csv')

## LSTM with random embedding model

In [5]:
%%time
train_data_path = '/nlp/cs_mixed_20k/cs_mixed_20k_train/'
test_data_path  = '/nlp/cs_mixed_20k/cs_mixed_20k_test/'
eval_without_intra_newline_path  = '/nlp/eval_without_intra_newline/'
eval_with_intra_newline_path     = '/nlp/eval_with_intra_newline/'

sentences_train, sent_tags_train = load_from_folder(train_data_path)
sentences_test, sent_tags_test = load_from_folder(test_data_path)
sentences_eval1, sent_tags_eval1 = load_from_folder(eval_without_intra_newline_path)
sentences_eval2, sent_tags_eval2 = load_from_folder(eval_with_intra_newline_path)

KeyboardInterrupt: 

In [6]:
xtrain, xvalid, ytrain, yvalid = train_test_split(sentences_train, sent_tags_train, test_size=0.1, random_state=42)

NameError: name 'sentences_train' is not defined

In [ ]:
feat_config = {'max_vocab':100000,
               'lstm_feats':True, 
               'spacy_feats':True, 
               'google_feats': False, 
               'neural_parscit_feats': False}

In [ ]:
%%time
featurizer = Featurizer(feat_config)
print("loading train ...")
train_dict, tokenizer, maxlen = featurizer.fit_transform(xtrain, ytrain)
print("loading valid ...")
valid_dict = featurizer.transform(xvalid, yvalid)
print("loading test ...")
test_dict  = featurizer.transform(sentences_test, sent_tags_test)
print("loading evals ...")
eval_dict1  = featurizer.transform(sentences_eval1, sent_tags_eval1)
eval_dict2  = featurizer.transform(sentences_eval2, sent_tags_eval2)

### Neural parscit features (temporary fix)

In [6]:
%%time
from feature.ParsCitLSTM import ParsCitLSTM
c = {
        "model_file": "/nlp/parscit/parscit-29-latest.h5",
        "label_dict_file": "/nlp/parscit/labels.json",
        "tfhub_model_dir": "/nlp/parscit/resource/"}
model = ParsCitLSTM(model_config=c)

CPU times: user 21.9 s, sys: 2.31 s, total: 24.2 s
Wall time: 23.7 s


In [7]:
%%time
train_df = pd.DataFrame([])
valid_df = pd.DataFrame([])
test_df = pd.DataFrame([])
eval1_df = pd.DataFrame([])
eval2_df = pd.DataFrame([])

train_df['text'] = np.array(xtrain, dtype='object')
valid_df['text'] = np.array(xvalid, dtype='object')
test_df['text'] = np.array(sentences_test, dtype='object')
eval1_df['text'] = np.array(sentences_eval1, dtype='object')
eval2_df['text'] = np.array(sentences_eval2, dtype='object')

CPU times: user 14.6 ms, sys: 0 ns, total: 14.6 ms
Wall time: 13.8 ms


In [13]:
%%time
#train_parscit = model.get_parscit_blocks(train_df)
#valid_parscit = model.get_parscit_blocks(valid_df)
#test_parscit  = model.get_parscit_blocks(test_df)
#eval1_parscit = model.get_parscit_blocks(eval1_df, chunk_size=40)
#eval2_parscit = model.get_parscit_blocks(eval2_df, chunk_size=40)

  0%|          | 0/34 [00:00<?, ?it/s]

  3%|▎         | 1/34 [00:53<29:33, 53.74s/it]

  6%|▌         | 2/34 [01:37<26:59, 50.60s/it]

  9%|▉         | 3/34 [02:13<23:53, 46.24s/it]

 12%|█▏        | 4/34 [03:00<23:19, 46.64s/it]

...

 91%|█████████ | 31/34 [24:01<02:15, 45.23s/it]

 94%|█████████▍| 32/34 [24:44<01:29, 44.67s/it]

 97%|█████████▋| 33/34 [25:32<00:45, 45.48s/it]

100%|██████████| 34/34 [25:42<00:00, 45.35s/it]

CPU times: user 23min 38s, sys: 5min 32s, total: 29min 10s
Wall time: 25min 42s


In [14]:
# train_parscit.to_pickle('/nlp/temp/train_parscit.pickle')
# valid_parscit.to_pickle('/nlp/temp/valid_parscit.pickle')
# test_parscit.to_pickle('/nlp/temp/test_parscit.pickle')
eval1_parscit.to_pickle('/nlp/temp/eval1_parscit.pickle')
eval2_parscit.to_pickle('/nlp/temp/eval2_parscit.pickle')

In [27]:
bla = pd.read_pickle('/nlp/temp/bla.pickle')

In [28]:
bla.head()

,parscit_feat
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [..."
1,"[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [..."
2,"[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [..."
3,"[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [..."
4,"[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [..."


In [14]:
for ind in range(len(train_parscit)):
    if len(xtrain[ind].split(" ")) != len(train_parscit.parscit_feat.iloc[ind]):
        print(ind)

## Keras with pre-trained fast text embedding

In [ ]:
model_config = {'bi-lstm':True,
                'lstm_units': 50,
                'trainable': True,
                'optimizer': Adam(
        learning_rate=0.001,
        beta_1=0.0,
        beta_2=0.0,
        epsilon=1e-05,
        amsgrad=False,
    ),
                'output_activation' : 'sigmoid', 
                'batch_size': 256
                
                
               }

In [ ]:
%%time
wiki_model = FastText.load_fasttext_format('/nlp/cc.en.300.bin')

In [ ]:
%%time
embedding_matrix = load_embedding_matrix(wiki_model, feat_config['max_vocab'], tokenizer.word_index, 300)

In [ ]:
%%time
input = Input(shape=(maxlen,))
embed = Embedding(input_dim=feat_config['max_vocab'], input_length=maxlen, output_dim=300, weights=[embedding_matrix], trainable=model_config['trainable'])(input)
lstm = LSTM(model_config['lstm_units'], return_sequences=True)(embed)
spacy_feats = Input(shape=(maxlen,12))
conc = Concatenate([lstm, spacy_feats])
td = TimeDistributed(Dense(1, activation=model_config['output_activation']))(lstm)
model = Model(inputs=[input, spacy_feats], outputs=td)
model.compile(optimizer=model_config['optimizer'],
    loss='binary_crossentropy',
    metrics=['accuracy']
)
print(model.summary())

In [ ]:
metrics = Metrics([train_dict['lstm_feats'], train_dict['labels']], [valid_dict['lstm_feats'], valid_dict['labels']])

In [ ]:
%%time
#spacy_train_arr = np.array([i.tolist() for i in train_dict['spacy_num_feats']])
#spacy_valid_arr = np.array([i.tolist() for i in valid_dict['spacy_num_feats']])
spacy_train_arr = train_dict['spacy_num_feats']
spacy_valid_arr = valid_dict['spacy_num_feats']
early_stop= EarlyStopping(monitor='val_loss',patience=3,verbose=0,mode='min',restore_best_weights=False, min_delta=0.0001)
history = model.fit([train_dict['lstm_feats'], spacy_train_arr], train_dict['labels'], verbose=1, epochs=5, batch_size= model_config['batch_size'], \
                    validation_data=([valid_dict['lstm_feats'], spacy_valid_arr], valid_dict['labels']), callbacks=[early_stop,metrics], shuffle=False)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
axes[0].plot(history.history['accuracy'])
axes[0].plot(history.history['val_accuracy'])
axes[0].title.set_text('model accuracy')
axes[0].set_ylabel('accuracy')
axes[0].set_xlabel('epoch')
axes[0].legend(['train', 'val'], loc='upper left')

axes[1].plot(history.history['loss'])
axes[1].plot(history.history['val_loss'])
axes[1].title.set_text('model loss')
axes[1].set_ylabel('loss')
axes[1].set_xlabel('epoch')
axes[1].legend(['train', 'val'], loc='upper left')
fig.tight_layout()

In [ ]:
print('done')

### Analysis on validation data

In [ ]:
%%time
valid_probs = model.predict([valid_dict['lstm_feats'], valid_dict['spacy_num_feats']])
valid_probs = valid_probs.reshape(valid_probs.shape[0], valid_probs.shape[1])
valid_preds = np.where(valid_probs > 0.5, 1, 0)

In [ ]:
%%time
result_valid = evaluate(valid_dict['labels'].tolist(), valid_preds.tolist())

In [ ]:
result_valid

In [ ]:
count_valid = 0
for ind in range(len(valid_preds)):
    pred = valid_preds[ind]
    true = valid_dict['labels'][ind]
    if (true == pred).all():
        pass
    else:
        count_valid += 1
        fp_ind = np.where((pred == 1) & (true == 0))[0]
        fn_ind = np.where((pred == 0) & (true == 1))[0]
        if len(fp_ind) > 0:
            #print(fp_ind)
            print(ind, 'False positives:', fp_ind)
            for x in fp_ind:
                #print(x)
                print(np.array(xvalid[ind].split(" "))[max(0, x-3):x+4])
        if len(fn_ind) > 0:
            print(ind, 'False negatives:', fn_ind)
            for x in fn_ind:
                print(np.array(xvalid[ind].split(" "))[max(0, x-3):x+4])

### Analysis of test result

In [ ]:
%%time
test_probs = model.predict([test_dict['lstm_feats'], test_dict['spacy_num_feats']])
test_probs = test_probs.reshape(test_probs.shape[0], test_probs.shape[1])
test_preds = np.where(test_probs > 0.5, 1, 0)

In [ ]:
%%time
result_test = evaluate(test_dict['labels'].tolist(), test_preds.tolist())

In [ ]:
result_test

In [ ]:
result_test

In [ ]:
tags = {'dataset':'cs_mixed_20k', 'data_split':'test'}
opt = model_config['optimizer']
model_config['optimizer'] = str(opt.get_config())
log_mlflow_results(model, result_test, feat_config, model_config, tags)

In [ ]:
count = 0
for ind in range(len(test_preds)):
    pred = test_preds[ind]
    true = new_y_test_enc[ind]
    if (true == pred).all():
        pass
    else:
        count += 1
        fp_ind = np.where((pred == 1) & (true == 0))[0]
        fn_ind = np.where((pred == 0) & (true == 1))[0]
        if len(fp_ind) > 0:
            #print(fp_ind)
            print(ind, 'False positives:', fp_ind)
            for x in fp_ind:
                #print(x)
                print(np.array(new_X_test[ind])[max(0, x-3):x+4])
        if len(fn_ind) > 0:
            print(ind, 'False negatives:', fn_ind)
            for x in fn_ind:
                print(np.array(new_X_test[ind])[max(0, x-3):x+4])

### Evaluation of eval dataset

In [1]:
keras_model = mlflow.keras.load_model("s3://caps-s3-mlflow/artifacts/4/e814b3fd2d854c1a83c5998cea627b59/artifacts/models")

NameError: name 'mlflow' is not defined

In [ ]:
eval_probs1 = model.predict([eval_dict1['lstm_feats'], eval_dict1['spacy_num_feats']])
eval_probs1 = eval_probs1.reshape(eval_probs1.shape[0], eval_probs1.shape[1])
eval_preds1 = np.where(eval_probs1 > 0.5, 1, 0)
result_eval1 = evaluate(eval_dict1['labels'].tolist(), eval_preds1.tolist())

In [ ]:
eval_probs2 = model.predict([eval_dict2['lstm_feats'], eval_dict2['spacy_num_feats']])
eval_probs2 = eval_probs2.reshape(eval_probs2.shape[0], eval_probs2.shape[1])
eval_preds2 = np.where(eval_probs2 > 0.5, 1, 0)
result_eval2 = evaluate(eval_dict2['labels'].tolist(), eval_preds2.tolist())

In [ ]:
result_eval2

In [ ]:
count_valid = 0
for ind in range(20):
    pred = eval_preds1[ind]
    true = eval_dict1['labels'][ind]
    if (true == pred).all():
        pass
    else:
        count_valid += 1
        fp_ind = np.where((pred == 1) & (true == 0))[0]
        fn_ind = np.where((pred == 0) & (true == 1))[0]
        if len(fp_ind) > 0:
            #print(fp_ind)
            print(ind, 'False positives:', fp_ind)
            for x in fp_ind:
                #print(x)
                print(np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4])
        if len(fn_ind) > 0:
            print(ind, 'False negatives:', fn_ind)
            for x in fn_ind:
                print(np.array(sentences_eval1[ind].split(" "))[max(0, x-3):x+4])

In [ ]:
tags = {'dataset':'cs_mixed_20k', 'data_split':'eval_with_intra_newline_path'}
#opt = model_config['optimizer']
#model_config['optimizer'] = str(opt.get_config())
log_mlflow_results(model, result_eval2, feat_config, model_config, tags)

In [ ]:
sentences_train[0][0:500]

In [ ]:
from feature.ParsCitLSTM import ParsCitLSTM

In [ ]:
#df = pd.read_csv('/nlp/parscit/data.csv')

In [ ]:
df = pd.DataFrame([])
df['text'] = np.array(sentences_train[0:100], dtype='object')

In [ ]:
%%time
df_out = model.dask_predict(df)

In [ ]:
#df_out

In [ ]:
# %%time
# import dask.dataframe as dd
# parscit_df = model.get_parscit_dask(df, blocksize=10)

In [ ]:
len(sentences_train[0].split(" "))

In [ ]:
%%time
x,y,z = model.predict(sentences_train[0])

In [ ]:
# for i,j in zip(x,y):
#     print(i,j)

### One predict test

In [ ]:
# model = load_model(c['model_file'])
# model._name='parscit_model'

In [ ]:
ls = np.random.randint(10, size=(1000))

In [ ]:
df = pd.DataFrame([], columns=['num'])
df['num'] = ls

In [15]:
x1 = np.random.rand(5,4,3)

In [32]:
x1

array([[[0.12696983, 0.96671784, 0.26047601],
        [0.89723652, 0.37674972, 0.33622174],
        [0.45137647, 0.84025508, 0.12310214],
        [0.5430262 , 0.37301223, 0.44799682]],

       [[0.12944068, 0.85987871, 0.82038836],
        [0.35205354, 0.2288873 , 0.77678375],
        [0.59478359, 0.13755356, 0.85289978],
        [0.23550748, 0.14622723, 0.58986877]],

       [[0.57401177, 0.06126996, 0.590426  ],
        [0.24534982, 0.34044494, 0.98472874],
        [0.9195404 , 0.03777169, 0.86154929],
        [0.75356885, 0.40517876, 0.34352588]],

       [[0.17091717, 0.39465901, 0.64166617],
        [0.27459243, 0.46235433, 0.87137165],
        [0.40113122, 0.61058827, 0.11796713],
        [0.70218436, 0.41403366, 0.34234521]],

       [[0.59592532, 0.19986426, 0.09973676],
        [0.73459622, 0.01654451, 0.4813845 ],
        [0.09593887, 0.49730633, 0.83879627],
        [0.89733326, 0.73259152, 0.75872436]]])

In [16]:
x2 = np.random.rand(5,4,2)

array([[[0.12696983, 0.96671784, 0.26047601, 0.56065718, 0.47147793],
        [0.89723652, 0.37674972, 0.33622174, 0.13876812, 0.09446113],
        [0.45137647, 0.84025508, 0.12310214, 0.94225634, 0.13409924],
        [0.5430262 , 0.37301223, 0.44799682, 0.63069955, 0.63956822]],

       [[0.12944068, 0.85987871, 0.82038836, 0.43348979, 0.15584706],
        [0.35205354, 0.2288873 , 0.77678375, 0.86260898, 0.6830463 ],
        [0.59478359, 0.13755356, 0.85289978, 0.0857089 , 0.56610253],
        [0.23550748, 0.14622723, 0.58986877, 0.14982485, 0.47745445]],

       [[0.57401177, 0.06126996, 0.590426  , 0.84893827, 0.14014442],
        [0.24534982, 0.34044494, 0.98472874, 0.32955081, 0.63407411],
        [0.9195404 , 0.03777169, 0.86154929, 0.61937849, 0.39504264],
        [0.75356885, 0.40517876, 0.34352588, 0.67239782, 0.80998032]],

       [[0.17091717, 0.39465901, 0.64166617, 0.38699906, 0.54475875],
        [0.27459243, 0.46235433, 0.87137165, 0.98894936, 0.98498291],
        [0.401